In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd


class Tabela:
    
    def __init__(self, nome_arquivo):
        self.nome_arquivo = nome_arquivo
    
    def atualizar(self):
        self.pesquisa_valores("dolar")
        self.pesquisa_valores("euro")
        self.pesquisa_valores("ouro")
        self.importa_tabela()
        display(self.tabela)
        self.atualiza_tabela()
        display(self.tabela)
        

    def pesquisa_valores(self, moeda):
        navegador = webdriver.Chrome()
        if moeda == "ouro":
                
                # Abre o site Melhor Câmbio
                navegador.get("https://www.melhorcambio.com/ouro-hoje")
                
                # Coleta o preço do ouro atualizado do site 
                cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
                
                # Formata para o padrão do Python
                cotacao_ouro = cotacao_ouro.replace(',','.')
                self.cotacao_ouro = cotacao_ouro
                
        elif moeda == "dolar":
                
                # Abre o Google e pesquisa pela cotação da moeda
                navegador.get("https://google.com.br/")
                navegador.find_element(By.XPATH, 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
                navegador.find_element(By.XPATH, 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
                
                # Coleta o valor da moeda atualizado e armazena na variável correspondente
                cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
                self.cotacao_dolar = cotacao_dolar
                    
        elif moeda == "euro":
            
                # Abre o Google e pesquisa pela cotação da moeda
                navegador.get("https://google.com.br/")
                navegador.find_element(By.XPATH, 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
                navegador.find_element(By.XPATH, 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
                
                # Coleta o valor da moeda atualizado e armazena na variável correspondente
                cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
                self.cotacao_euro = cotacao_euro
            
    def importa_tabela(self):
        tabela = pd.read_excel(self.nome_arquivo)
        self.tabela = tabela
    
    def atualiza_tabela(self):
        self.tabela.loc[self.tabela["Moeda"] == "Dólar", "Cotação"] = float(self.cotacao_dolar)
        self.tabela.loc[self.tabela["Moeda"] == "Euro", "Cotação"] = float(self.cotacao_euro)
        self.tabela.loc[self.tabela["Moeda"] == "Ouro", "Cotação"] = float(self.cotacao_ouro)
        self.tabela["Preço de Compra"] = self.tabela["Preço Original"] * self.tabela["Cotação"]
        self.tabela["Preço de Venda"] = self.tabela["Preço de Compra"] * self.tabela["Margem"]
        self.tabela["Preço de Venda"] = self.tabela["Preço de Venda"].map("R${:.2f}".format)
        self.tabela.to_excel("Produtos Novo.xlsx", index=False)
    
    
    
    
tentativa = Tabela("Produtos.xlsx")
tentativa.atualizar()

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.98560,4985.550144,1.40,R$6979.77
1,Carro Renault,4500.00,Euro,5.21758,23479.109856,2.00,R$46958.22
2,Notebook Dell,899.99,Dólar,4.98560,4486.990144,1.70,R$7627.88
3,IPhone,799.00,Dólar,4.98560,3983.494400,1.70,R$6771.94
4,Carro Fiat,3000.00,Euro,5.21758,15652.739904,1.90,R$29740.21
5,Celular Xiaomi,480.48,Dólar,4.98560,2395.481088,2.00,R$4790.96
6,Joia 20g,20.00,Ouro,290.91000,5818.200000,1.15,R$6690.93
